# अध्याय 3 मूल्यांकन इनपुट - वर्गीकरण

- [आई. पर्यावरण विन्यास](#आई. पर्यावरण विन्यास)
- [II. उपयोगकर्ता निर्देशों को वर्गीकृत करें](#II. उपयोगकर्ता निर्देशों को वर्गीकृत करें)

इस अध्याय में, हम इनपुट कार्यों के मूल्यांकन पर ध्यान केंद्रित करेंगे, जो सिस्टम की गुणवत्ता और सुरक्षा सुनिश्चित करने के लिए महत्वपूर्ण है।

ऐसे कार्यों के लिए जिनमें विभिन्न स्थितियों में निर्देशों के कई स्वतंत्र सेटों को संभालने की आवश्यकता होती है, पहले क्वेरी प्रकार को वर्गीकृत करना और इसे आधार के रूप में उपयोग करना फायदेमंद होता है ताकि यह निर्धारित किया जा सके कि कौन से निर्देशों का उपयोग करना है।

इसे निश्चित श्रेणियों को परिभाषित करके और उन निर्देशों को हार्ड-कोड करके प्राप्त किया जा सकता है जो किसी दिए गए श्रेणी में कार्यों को संभालने के लिए प्रासंगिक हैं।

उदाहरण के लिए, जब बूग्राहक सेवा सहायक बनाते समय, पहले क्वेरी प्रकार को वर्गीकृत करना और फिर उस वर्गीकरण के आधार पर उपयोग के लिए निर्देश निर्धारित करना महत्वपूर्ण हो सकता है।

एक विशिष्ट उदाहरण के रूप में, यदि उपयोगकर्ता अपना खाता बंद करने के लिए कहता है, तो द्वितीयक निर्देश खाता बंद करने के तरीके पर अतिरिक्त निर्देश जोड़ने का हो सकता है, जबकि यदि उपयोगकर्ता किसी विशिष्ट उत्पाद के बारे में जानकारी मांगता है, तो द्वितीयक निर्देश यह हो सकता है; अधिक उत्पाद जानकारी जोड़ें.

## 1. पर्यावरण विन्यास
पिछले अध्याय की तरह, हमें सबसे पहले OpenAI API का उपयोग करने के लिए पर्यावरण को कॉन्फ़िगर करना होगा

In [9]:
import openai
# तृतीय-पक्ष लाइब्रेरी आयात करें

openai.api_key  = "sk-..."
# API_KEY सेट करें, कृपया इसे अपनी API_KEY से बदलें

# निम्नलिखित पर्यावरण चर पर आधारित कॉन्फ़िगरेशन विधि का एक उदाहरण है, जो अधिक सुरक्षित है और इसे बाद में कवर नहीं किया जाएगा।
#आयात openai
# ओएस आयात करें
# OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
# openai.api_key = OPENAI_API_KEY

In [2]:
def get_completion_from_messages(messages, 
                                model="gpt-3.5-turbo", 
                                temperature=0, 
                                max_tokens=500):
    '''
    封装一个访问 OpenAI GPT3.5 的函数

    参数: 
    messages: 这是一个消息列表，每个消息都是一个字典，包含 role(角色）和 content(内容)。角色可以是'system'、'user' 或 'assistant’，内容是角色的消息。
    model: 调用的模型，默认为 gpt-3.5-turbo(ChatGPT)，有内测资格的用户可以选择 gpt-4
    temperature: 这决定模型输出的随机程度，默认为0，表示输出将非常确定。增加温度会使输出更随机。
    max_tokens: 这决定模型输出的最大的 token 数。
    '''
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # 这决定模型输出的随机程度
        max_tokens=max_tokens, # 这决定模型输出的最大的 token 数
    )
    return response.choices[0].message["content"]

## 2. उपयोगकर्ता निर्देशों को वर्गीकृत करें

यहां, हम सिस्टम के लिए वैश्विक गाइड के रूप में system_message का उपयोग करते हैं और सीमांकक के रूप में `#` चुनते हैं।

डिलीमीटर एक उपकरण है जिसका उपयोग कमांड या आउटपुट के विभिन्न भागों को अलग करने के लिए किया जाता है। यह मॉडल को भागों की पहचान करने में मदद करता है, जिससे विशिष्ट कार्यों को करने में सिस्टम की सटीकता और दक्षता में सुधार होता है।

इस उदाहरण में, हम सीमांकक के रूप में `#` का उपयोग करना चुनते हैं।

`#` एक आदर्श सीमांकक है क्योंकि इसे एक स्वतंत्र टोकन के रूप में माना जा सकता है।

In [3]:
delimiter = "####"

यह हमारा सिस्टम संदेश है और हम मॉडल से निम्नलिखित तरीके से पूछ रहे हैं।

In [4]:
system_message = f"""
You will be provided with customer service queries. \
The customer service query will be delimited with \
{delimiter} characters.
Classify each query into a primary category \
and a secondary category. 
Provide your output in json format with the \
keys: primary and secondary.

Primary categories: Billing, Technical Support, \
Account Management, or General Inquiry.

Billing secondary categories:
Unsubscribe or upgrade
Add a payment method
Explanation for charge
Dispute a charge

Technical Support secondary categories:
General troubleshooting
Device compatibility
Software updates

Account Management secondary categories:
Password reset
Update personal information
Close account
Account security

General Inquiry secondary categories:
Product information
Pricing
Feedback
Speak to a human

"""

In [5]:
system_message = f"""
你将获得客户服务查询。
每个客户服务查询都将用{delimiter}字符分隔。
将每个查询分类到一个主要类别和一个次要类别中。
以 JSON 格式提供你的输出，包含以下键：primary 和 secondary。

主要类别：计费（Billing）、技术支持（Technical Support）、账户管理（Account Management）或一般咨询（General Inquiry）。

计费次要类别：
取消订阅或升级（Unsubscribe or upgrade）
添加付款方式（Add a payment method）
收费解释（Explanation for charge）
争议费用（Dispute a charge）

技术支持次要类别：
常规故障排除（General troubleshooting）
设备兼容性（Device compatibility）
软件更新（Software updates）

账户管理次要类别：
重置密码（Password reset）
更新个人信息（Update personal information）
关闭账户（Close account）
账户安全（Account security）

一般咨询次要类别：
产品信息（Product information）
定价（Pricing）
反馈（Feedback）
与人工对话（Speak to a human）

"""

आइए अब उपयोगकर्ता संदेश का एक उदाहरण देखें।

In [26]:
user_message = f"""\ 
I want you to delete my profile and all of my user data"""

In [6]:
user_message = f"""\ 
我希望你删除我的个人资料和所有用户数据。"""

इस संदेश को संदेशों की एक सूची के रूप में स्वरूपित करें, सिस्टम संदेशों और उपयोगकर्ता संदेशों को "####" से अलग करें।

आइए एक इंसान के रूप में सोचें कि यह वाक्य किस श्रेणी का है: "मैं चाहता हूं कि आप मेरी प्रोफ़ाइल हटा दें।"

यह वाक्य ऐसा लगता है जैसे यह "खाता प्रबंधन" या "खाता बंद करें" से संबंधित है।

In [7]:
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
]

आइए देखें कि मॉडल कैसे सोचती है

मॉडल का वर्गीकरण "खाता प्रबंधन" को "प्राथमिक" और "खाता बंद करें" को "द्वितीयक" के रूप में वर्गीकृत किया गया है।

JSON जैसे संरचित आउटपुट का अनुरोध करने का लाभ यह है कि आप इसे आसानी से किसी ऑब्जेक्ट में पढ़ सकते हैं, जैसे कि पायथन में एक शब्दकोश। यदि आप किसी अन्य भाषा का उपयोग करते हैं, तो आप इसे किसी अन्य ऑब्जेक्ट में भी परिवर्तित कर सकते हैं और फिर इसे बाद के चरणों में इनपुट कर सकते हैं।

In [10]:
response = get_completion_from_messages(messages)
print(response)

{
    "primary": "账户管理",
    "secondary": "关闭账户"
}


यहां एक अन्य उपयोगकर्ता संदेश है: "मुझे अपने फ्लैट स्क्रीन टीवी के बारे में और बताएं"

हम मॉडल की प्रतिक्रिया प्राप्त करने और उसका प्रिंट आउट लेने के लिए संदेशों की वही सूची लागू करते हैं।

एक अन्य वर्गीकरण परिणाम यहां दिया गया है और ऐसा लगता है कि यह सही होना चाहिए।

In [31]:
user_message = f"""\
Tell me more about your flat screen tvs"""
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 
response = get_completion_from_messages(messages)
print(response)

{
  "primary": "General Inquiry",
  "secondary": "Product information"
}


In [12]:
user_message = f"""\
告诉我更多有关你们的平板电脑的信息"""
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 
response = get_completion_from_messages(messages)
print(response)

以下是针对平板电脑的一般咨询：

{
  "primary": "General Inquiry",
  "secondary": "Product information"
}

如果您有任何特定的问题或需要更详细的信息，请告诉我，我会尽力回答。


इसलिए, ग्राहक पूछताछ के वर्गीकरण के आधार पर, अब हम अगले चरणों को संभालने के लिए निर्देशों का अधिक विशिष्ट सेट प्रदान कर सकते हैं।

इस मामले में, हम टीवी के बारे में अतिरिक्त जानकारी जोड़ सकते हैं, जबकि अन्य मामलों में, हम खाता बंद करने के लिए एक लिंक या कुछ इसी तरह प्रदान करना चाह सकते हैं।

निम्नलिखित अध्यायों में, हम इनपुट को संभालने के विभिन्न तरीकों पर करीब से नज़र डालेंगे

अगले अध्याय में, हम इनपुट का मूल्यांकन करने के तरीकों के बारे में और अधिक जानकारी प्राप्त करेंगे, विशेष रूप से यह कैसे सुनिश्चित करेंलोग सिस्टम का जिम्मेदार तरीके से उपयोग करते हैं।